<a href="https://colab.research.google.com/github/davis689/binder/blob/master/Keq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sympy import *
from scipy.optimize import brentq,fsolve
import matplotlib.pyplot as plt
import requests
import lxml.html as lh
import pandas as pd

Set up

In [2]:
T,V,A,B,p,x,xi=var('T, V,A,B,p,x,xi')
R=8.3144598 

Import data from webbook.nist.gov for each molecule

In [3]:
url = 'https://webbook.nist.gov/cgi/cbook.cgi?ID=C10024972&Units=SI&Mask=1#Thermo-Gas' #N2O
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-3] # change index to get different table. -1 means start at end
So=[]
So.append(float(df.iloc[2,1]))
df=df_list[-2]
tmp=df.iloc[1:9,1].values.tolist()
coef=[]
coef.append([float(tmp[i]) for i in range(len(tmp))])

In [4]:
url = 'https://webbook.nist.gov/cgi/cbook.cgi?ID=C7782447&Units=SI&Mask=1#Thermo-Gas' #O2
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-3] # change index to get different table. -1 means start at end
So.append(float(df.iloc[1,1]))
df=df_list[-2]
tmp=df.iloc[1:9,1].values.tolist()
coef.append([float(tmp[i]) for i in range(len(tmp))])

In [5]:
url = 'https://webbook.nist.gov/cgi/cbook.cgi?ID=C10102440&Units=SI&Mask=1#Thermo-Gas' #NO2
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-3] # change index to get different table. -1 means start at end
So.append(float(df.iloc[2,1]))
df=df_list[-2]
tmp=df.iloc[1:9,1].values.tolist()
coef.append([float(tmp[i]) for i in range(len(tmp))])

Coefficients and the powers in the Shomate equation.

In [6]:
stoic=[-2,-3,4] # stoichiometry with reactant negative.

In [7]:
#@title Input the temperature and pressure at which you want to calculate entropy change
pi = 1
pf =  0.5#@param {type:"number"}
Ti = 298
Tf =  400#@param {type:"number"}
n_N2O =  1#@param {type:"number"}
n_O2 =  1#@param {type:"number"}
n_NO2 =  1#@param {type:"number"}
nrxn=[n_N2O,n_O2,n_NO2]

In [8]:
Hf=[sum([coef[i][j]/(j+1)*(T/1000)**(j+1) for j in range(4)]) for i in range(len(stoic))]
Hf=[Hf[i]-coef[i][4]*(1000/T)+coef[i][5]-coef[i][7] for i in range(len(stoic))] # temperature change for each species
DH=sum([Hf[i]*stoic[i] for i in range(len(stoic))])
DH.subs(T,Tf)

Hfo=sum([coef[i][7]*stoic[i] for i in range(len(stoic))]) # standard reaction enthalpy change

DHtot=DH+Hfo
DHtot.subs(T,Tf).round(2)

-33.38

In [38]:
S=[sum([coef[i][j]/(j)*(T/1000)**j for j in range(4) if j>0]) for i in range(len(stoic))] # take care of the polynomial part of the equation
S=[S[i]+coef[i][0]*ln(T/1000)-coef[i][4]/(2*(T/1000)**2)+coef[i][6] for i in range(len(stoic))] # add terms for the non-polynomial terms
DStot=sum([S[i]*stoic[i] for i in range(len(stoic))]) # add up the entropies for all species with the appropriate stoichiometry
DStot.subs(T,Tf).n().round(2) # substitute for temperature to get a number 
DStot_p=DStot+R*ln(pf) #If we account for pressure here. Below for pressure accounted for in the delta G equation.
print(DStot.subs(T,Tf).round(2),DStot_p.subs(T,Tf).round(2),(DHtot*1000-Tf*DStot_p).subs(T,Tf).round(2)) # calc DS for reaction and temperature change, for rxn, T, and p, and DG for same

-100.05 -105.82 8942.17


In [23]:
print(So)
sum([So[i]*stoic[i] for i in range(len(stoic))]) #standard entropy change at 298 K and 1 bar.

[219.96, 205.15, 240.04]


-95.21000000000015

In [12]:
DG=DHtot*1000-T*DStot+sum(stoic)*R*T*log(pf)
DG.subs(T,Tf).n().round(2) # DG... same as above, calculated differently

8942.17

In [13]:
Keq=exp(-DG/R/T)
Keq.subs(T,Tf) #calculate Keq

0.0679665679167046

In [14]:
ch=[nrxn[i]+stoic[i]*xi for i in range(len(stoic))]; sm=sum(ch) #calculate moles with xi and sum them up.
num=[(ch[i]/sm)**stoic[i] for i in range(len(stoic)) if stoic[i]>0] #the numerator in Keq
den=[(ch[i]/sm)**stoic[i] for i in range(len(stoic)) if stoic[i]<0] #the denominator in Keq. Both are not needed except that the Keq solver works better if we don't use a complicated fraction.

In [15]:
rangh=[-nrxn[i]/stoic[i] for i in range(len(stoic)) if stoic[i]>0]
rangl=[-nrxn[i]/stoic[i] for i in range(len(stoic)) if stoic[i]<0]
ximin=min(rangh)
ximax=min(rangl)
print(ximin,ximax)

-0.25 0.3333333333333333


In [16]:
import operator
import functools
def prod(factors): #make a function to take the product of a bunch of numbers.
    return functools.reduce(operator.mul, factors, 1)

In [17]:
def K(x):
  num=[(ch[i]/sm).subs(xi,x)**stoic[i] for i in range(len(stoic)) if stoic[i]>0]
  den=[(ch[i]/sm).subs(xi,x)**-stoic[i] for i in range(len(stoic)) if stoic[i]<0]
  return prod(num)-Keq.subs(T,Tf)*prod(den) # use prod to make Keq but make only numerator and denominator separately. Multiply Keq calculated from DG and mult by the denominator.

In [18]:
x=brentq(K,ximin,ximax) #find zeros of the function above between the minimum and maximum possible xi.
x

-0.11644713594018029

In [19]:
n=[ch[i].subs(xi,x) for i in range(len(stoic))] # substitute xi into moles expressions to get answer.
n

[1.23289427188036, 1.34934140782054, 0.534211456239279]

In [20]:
n[2]**stoic[2]*n[1]**stoic[1]*n[0]**stoic[0]*sm.subs(xi,x)-Keq.subs(T,Tf)<10**-8

True